In [1]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from liblinear.liblinearutil import *

In [2]:
datContentTrain = [i.strip().split() for i in open("./hw4_train.dat").readlines()]
for i in range(len(datContentTrain)):
    for j in range(len(datContentTrain[i])):
        datContentTrain[i][j] = float(datContentTrain[i][j])
data_train = np.array(datContentTrain)

datContentTest = [i.strip().split() for i in open("./hw4_test.dat").readlines()]
for i in range(len(datContentTest)):
    for j in range(len(datContentTest[i])):
        datContentTest[i][j] = float(datContentTest[i][j])
data_test = np.array(datContentTest)

In [3]:
X_train = data_train[:, :-1]
y_train = data_train[:, -1]
X_test = data_test[:, :-1]
y_test = data_test[:, -1]

In [4]:
poly = PolynomialFeatures(3)
X_train_transformed_Q3 = poly.fit_transform(X_train)
X_test_transformed_Q3 = poly.transform(X_test)

# Problem 12

## By reading the documentation, we can derive that $\lambda = \frac{1}{2C}$

In [19]:
#algo
lambda_list = [-4, -2, 0, 2, 4]
for lam in lambda_list:
    print(f"When log_10 (lambda) = {lam}")
    prob = problem(y_train, X_train_transformed_Q3)
    c = 1 / (2 * (10 ** lam))
    param = parameter(f"-s 0 -c {c} -e 0.000001 -q")
    m = train(prob, param)
    p_labels, p_acc, p_vals = predict(y_test, X_test_transformed_Q3, m)
    
    (ACC, MSE, SCC) = evaluations(y_test, p_labels)

When log_10 (lambda) = -4
Accuracy = 87.375% (699/800) (classification)
When log_10 (lambda) = -2
Accuracy = 87.75% (702/800) (classification)
When log_10 (lambda) = 0
Accuracy = 92% (736/800) (classification)
When log_10 (lambda) = 2
Accuracy = 92.75% (742/800) (classification)
When log_10 (lambda) = 4
Accuracy = 83.5% (668/800) (classification)


# Problem 13

In [26]:
#algo
lambda_list = [-4, -2, 0, 2, 4]
for lam in lambda_list:
    print(f"When log_10 (lambda) = {lam}")
    prob = problem(y_train, X_train_transformed_Q3)
    c = 1 / (2 * (10 ** lam))
    param = parameter(f"-s 0 -c {c} -e 0.000001 -q")
    m = train(prob, param)
    p_labels, p_acc, p_vals = predict(y_train, X_train_transformed_Q3, m)

When log_10 (lambda) = -4
Accuracy = 100% (200/200) (classification)
When log_10 (lambda) = -2
Accuracy = 100% (200/200) (classification)
When log_10 (lambda) = 0
Accuracy = 97.5% (195/200) (classification)
When log_10 (lambda) = 2
Accuracy = 95% (190/200) (classification)
When log_10 (lambda) = 4
Accuracy = 82% (164/200) (classification)


# Problem 14

In [7]:
X_subtrain = X_train_transformed_Q3[:120, :]
y_subtrain = y_train[:120]
X_val = X_train_transformed_Q3[120:, :]
y_val = y_train[120:]

In [82]:
lambda_list = [-4, -2, 0, 2, 4]
best_lambda = 0
best_acc = 0
best_model = None
for lam in lambda_list:
    print(f"When log_10 (lambda) = {lam}")
    prob = problem(y_subtrain, X_subtrain)
    c = 1 / (2 * (10 ** lam))
    param = parameter(f"-s 0 -c {c} -e 0.000001 -q")
    m = train(prob, param)
    p_labels, p_acc, p_vals = predict(y_val, X_val, m)
    if p_acc[0] > best_acc:
        best_lambda = lam
        best_acc = p_acc[0]
        best_model = m

print("="*10)
print(f"The best lambda selected by sub-training set is {best_lambda}")
p_labels, p_acc, p_vals = predict(y_test, X_test_transformed_Q3, best_model, "-q")
print(f"Testing result: E_out is {1 - p_acc[0] / 100}")

When log_10 (lambda) = -4
Accuracy = 77.5% (62/80) (classification)
When log_10 (lambda) = -2
Accuracy = 80% (64/80) (classification)
When log_10 (lambda) = 0
Accuracy = 88.75% (71/80) (classification)
When log_10 (lambda) = 2
Accuracy = 93.75% (75/80) (classification)
When log_10 (lambda) = 4
Accuracy = 85% (68/80) (classification)
The best lambda selected by sub-training set is 2
Testing result: E_out is 0.07499999999999996


# Problem 15

In [57]:
#best_lambda = 2 from previous problem
prob = problem(y_train, X_train_transformed_Q3)
c = 1 / (2 * (10 ** best_lambda))
param = parameter(f"-s 0 -c {c} -e 0.000001 -q")
m = train(prob, param)
p_labels, p_acc, p_vals = predict(y_test, X_test_transformed_Q3, m)
print(f"Testing result: E_out is {1 - p_acc[0] / 100}")

Accuracy = 92.75% (742/800) (classification)
Testing result: E_out is 0.07250000000000001


# Problem 16

In [10]:
num_split = 40
K = int(X_train.shape[0] / num_split) #K-Fold cv

for lam in lambda_list:
    ttl_acc = 0
    for idx in range(K):
        X_val = X_train_transformed_Q3[idx*num_split:(idx+1)*num_split, :]
        y_val = y_train[idx*num_split:(idx+1)*num_split]
        X_subtrain = np.concatenate(
            [X_train_transformed_Q3[:idx*num_split, :], 
            X_train_transformed_Q3[(idx+1)*num_split:, :]],
            axis = 0)
        y_subtrain = np.concatenate(
            [y_train[:idx*num_split], 
            y_train[(idx+1)*num_split:]],
            axis = 0
        )

        prob = problem(y_subtrain, X_subtrain)
        c = 1 / (2 * (10 ** lam))
        param = parameter(f"-s 0 -c {c} -e 0.000001 -q")
        m = train(prob, param)
        p_labels, p_acc, p_vals = predict(y_val, X_val, m, "-q")

        ttl_acc += p_acc[0]
    print(f"When lambda = {lam}, cross validation error is {1 - ttl_acc/(K*100)}")

When lambda = -4, cross validation error is 0.18999999999999995
When lambda = -2, cross validation error is 0.17000000000000004
When lambda = 0, cross validation error is 0.11499999999999999
When lambda = 2, cross validation error is 0.07999999999999996
When lambda = 4, cross validation error is 0.20999999999999996
